# Imports and Setup
Python in Finance: Real Time Data Streaming within Jupyter Notebook
Credit: https://www.kdnuggets.com/python-in-finance-real-time-data-streaming-within-jupyter-notebook
https://colab.research.google.com/github/pathwaycom/pathway-examples/blob/main/showcases/live-data-jupyter.ipynb

In [1]:
%%capture --no-display
!pip install pathway

In [2]:
# Importing libraries for data processing, visualization, and dashboard creation

import datetime
import bokeh.models
import bokeh.plotting
import panel
import pathway as pw

# Fetching Sample Data

In [3]:
# Command to download the sample APPLE INC stock prices extracted via Polygon API and stored in a CSV for ease of review of this notebook.

#%%capture --no-display
!wget -nc https://gist.githubusercontent.com/janchorowski/e351af72ecd8d206a34763a428826ab7/raw/ticker.csv

--2024-02-23 01:07:08--  https://gist.githubusercontent.com/janchorowski/e351af72ecd8d206a34763a428826ab7/raw/ticker.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253370 (1.2M) [text/plain]
Saving to: 'ticker.csv'

ticker.csv          100%[===================>]   1.19M  --.-KB/s    in 0.04s   

2024-02-23 01:07:09 (32.3 MB/s) - 'ticker.csv' saved [1253370/1253370]



# Data Source Setup

In [4]:
# Creating a streaming data source from a CSV file

fname = "ticker.csv"
schema = pw.schema_from_csv(fname)
data = pw.demo.replay_csv(fname, schema=schema, input_rate=1000)

# Uncommenting the line below will override the data table defined above and switch the data source to static mode, which is helpful for initial testing
# data = pw.io.csv.read(fname, schema=schema, mode="static")

# Parsing the timestamps in the data

data = data.with_columns(t=data.t.dt.utc_from_timestamp(unit="ms"))

# Calculating the stats essential for Bollinger Bands

In [5]:
# Calculating the 20-minute rolling statistics for Bollinger Bands


minute_20_stats = (
    data.windowby(
        pw.this.t,
        window=pw.temporal.sliding(
            hop=datetime.timedelta(minutes=1),
            duration=datetime.timedelta(minutes=20),
        ),
        behavior=pw.temporal.exactly_once_behavior(),
        instance=pw.this.ticker,
    )
    .reduce(
        ticker=pw.this._pw_instance,
        t=pw.this._pw_window_end,
        volume=pw.reducers.sum(pw.this.volume),
        transact_total=pw.reducers.sum(pw.this.volume * pw.this.vwap),
        transact_total2=pw.reducers.sum(pw.this.volume * pw.this.vwap**2),
    )
    .with_columns(vwap=pw.this.transact_total / pw.this.volume)
    .with_columns(
        vwstd=(pw.this.transact_total2 / pw.this.volume - pw.this.vwap**2)
        ** 0.5
    )
    .with_columns(
        bollinger_upper=pw.this.vwap + 2 * pw.this.vwstd,
        bollinger_lower=pw.this.vwap - 2 * pw.this.vwstd,
    )
)
# Computing the 1-minute rolling statistics

minute_1_stats = (
    data.windowby(
        pw.this.t,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=1)),
        behavior=pw.temporal.exactly_once_behavior(),
        instance=pw.this.ticker,
    )
    .reduce(
        ticker=pw.this._pw_instance,
        t=pw.this._pw_window_end,
        volume=pw.reducers.sum(pw.this.volume),
        transact_total=pw.reducers.sum(pw.this.volume * pw.this.vwap),
    )
    .with_columns(vwap=pw.this.transact_total / pw.this.volume)
)
# Joining the 1-minute and 20-minute statistics for comprehensive analysis

joint_stats = (
    minute_1_stats.join(
        minute_20_stats,
        pw.left.t == pw.right.t,
        pw.left.ticker == pw.right.ticker,
    )
    .select(
        *pw.left,
        bollinger_lower=pw.right.bollinger_lower,
        bollinger_upper=pw.right.bollinger_upper
    )
    .with_columns(
        is_alert=(pw.this.volume > 10000)
        & (
            (pw.this.vwap > pw.this.bollinger_upper)
            | (pw.this.vwap < pw.this.bollinger_lower)
        )
    )
    .with_columns(
        action=pw.if_else(
            pw.this.is_alert,
            pw.if_else(
                pw.this.vwap > pw.this.bollinger_upper, "sell", "buy"
            ),
            "hold",
        )
    )
)
alerts = joint_stats.filter(pw.this.is_alert)

# Dashboard Creation

In [6]:
# Function to create the statistics plot


def stats_plotter(src):
    actions = ["buy", "sell", "hold"]
    color_map = bokeh.models.CategoricalColorMapper(
        factors=actions, palette=("#00ff00", "#ff0000", "#00000000")
    )

    fig = bokeh.plotting.figure(
        height=400,
        width=600,
        title="20 minutes Bollinger bands with last 1 minute average",
        x_axis_type="datetime",
        y_range=(188.5, 191),
    )
    fig.line("t", "vwap", source=src)
    band = bokeh.models.Band(
        base="t",
        lower="bollinger_lower",
        upper="bollinger_upper",
        source=src,
        fill_alpha=0.3,
        fill_color="gray",
        line_color="black",
    )
    fig.scatter(
        "t",
        "vwap",
        color={"field": "action", "transform": color_map},
        size=10,
        marker="circle",
        source=src,
    )
    fig.add_layout(band)
    return fig


# Combining the plot and table in a Panel Row

viz = panel.Row(
    joint_stats.plot(stats_plotter, sorting_col="t"),
    alerts.select(
        pw.this.ticker, pw.this.t, pw.this.vwap, pw.this.action
    ).show(include_id=False, sorters=[{"field": "t", "dir": "desc"}]),
)
viz

Row
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Tabulator(disabled=True, height=400, show_index=False, sorters=[{'field': 't', ...], value=Empty DataFrame
Columns: [...)

# Running the Computation

In [7]:
# Command to start the Pathway data processing engine
#%%capture --no-display
pw.run()

Output()

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

...

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

/opt/conda/lib/python3.10/site-packages/pathway/stdlib/viz/table_viz.py:140: FutureWarning: DataFrame.applymap has 
been deprecated. Use DataFrame.map instead.
  df = df.applymap(_format_types)  # type: ignore

# Due to the bug described in https://github.com/jupyter/notebook/issues/1622
# Please execute `pw.run()` manually, after the widget above is initialized and visible.

# pw.run()